# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science

## Project Group #26: Lending Club


**Harvard University**<br/>
**Fall 2018**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader<br/>
**TF**: Jerry Peng<br/>
**Students**: Michel Atoudem Kana, William Groves, Shourya Veeraganti

<hr style="height:2pt">



# Data Description and Cleaning

In [348]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

import numpy as np
import pandas as pd


# Data Description

Lending Club publishes all its historical data since its inception in 2007. It provides two online, open access datasets for accepted and rejected loans from 2007 to 2018 Q2 for a period of 11.5 years in comma-separated values (CSV) format. Each accepted loan data set has 146 features for each of observation. With basic data cleaning, we removed the index column, columns with constant value and columns associated with 90% missing values that reduced the number of features to 106. Further we identified features which are strongly correlated (r=±0.8) that left us with 82 features on the accepted loan dataset.

To help us in our goal of analyzing fairness and interpretability we downloaded the census data about American people and US economy from United States Census Bureau. We have social, economic, and geographic data from 2016 grouped by zip code provided via Piazza by CS109a instructors. This dataset has 33120 zip codes described by 135 features. Trivial columns such as name, population and others were removed that left us with 85 features. Then the census data was merged with loan accepted data on zipcode ending with 167 features. Then we further used scikit learn feature selection on SelectPercentile reducing 141 predictors for our model analysis. For reconciliation, we downloaded data for all years and then we took a random sample of 10% and saved it to our own server “https://digintu.tech/tmp/cs109a/” stored in the file “loan_accepted_10.csv” for easy access. Also removed empty/duplicate rows that left us with 200K observations.

Second dataset provided online is the rejected loan information, again from 2007 to 2018 Q2 for 11.5 years. This dataset has 9 features. Each application of rejected loan has 9 features and the total amount was worth $22million not funded. These Rejected Loans data files contain the list and details of all loan applications that did not meet Lending Club's credit underwriting policy and the application was rejected.

We group the remaining features from loan accepted merged with census data into 5 classes:
● loan data: information about the loan at the moment when it was requested
● loan follow up: information about the loan's follow up throughout its term
● borrower demographics: information about the borrower
● borrower financial profile: financial background of the borrower at the moment when he requested the loan
● borrower financial profile follow up: changes in financial profile of the borrower throughout the loan term

# Raw Data

**Helper functions**

In [395]:
"""
The following function returns the description of the features in lending club data.
Parameters
  df: dataframe containing the data
"""
def df_features_desc(df):
    df_cols_desc = load_data(['LCDataDictionary.csv'])
    desc = pd.DataFrame(df.columns).merge(df_cols_desc, how='left', left_on=0, right_on='LoanStatNew')[['LoanStatNew','Description']]
    desc = pd.DataFrame(df.dtypes).reset_index().merge(desc, how='right', left_on='index', right_on='LoanStatNew')[['LoanStatNew', 0,'Description']]
    desc = desc.rename(columns={0: 'Data Type', 'LoanStatNew':'Feature'})#.sort_values(by=['Feature'])
    desc = desc.dropna()
    pd.set_option('display.max_colwidth', -1)
    desc.style.set_properties({'text-align': 'left'})
    display(HTML(desc.to_html()))

## Accepted Loans

We load all loans accepted (approved) and funded on the LendingClub marketplace from 2007 to 2018 Q3 for a period of 11.5 years from separated files into the dataframe `df_loan_accepted`.

In [350]:
df_loan_accepted = load_data(['loan_accepted_10.csv'])

In [246]:
df_loan_accepted.shape

(200409, 146)

There are approximately **2 millions accepted loans**, each of them has 146 columns. Many columns are however empty.

In [7]:
df_loan_accepted.describe()

,Unnamed: 0,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,200409.000000,0.0,200407.000000,200407.000000,200407.000000,200407.000000,2.004060e+05,0.0,200293.000000,200402.000000,...,881.0,881.000000,881.0,881.000000,700.000000,881.000000,881.000000,2619.000000,2619.000000,2619.000000
mean,99055.357324,NaN,14925.943330,14920.633261,14899.501524,443.203570,7.742113e+04,NaN,18.653801,0.315266,...,3.0,150.793394,3.0,12.542565,443.717143,11555.922758,200.722781,5043.492249,47.868813,12.512791
std,91099.127758,NaN,9046.369332,9044.458980,9048.247653,264.271981,7.869853e+04,NaN,12.279450,0.869024,...,0.0,117.160448,0.0,10.002764,343.897939,7079.323715,197.927608,3650.713791,7.220576,8.330692
min,0.000000,NaN,500.000000,500.000000,0.000000,16.850000,0.000000e+00,NaN,0.000000,0.000000,...,3.0,0.640000,3.0,0.000000,1.920000,55.730000,0.010000,221.260000,20.000000,0.000000
25%,36044.000000,NaN,8000.000000,8000.000000,8000.000000,251.680000,4.600000e+04,NaN,11.920000,0.000000,...,3.0,62.200000,3.0,0.000000,183.735000,6245.200000,40.040000,2265.470000,45.000000,6.000000
50%,74274.000000,NaN,12800.000000,12800.000000,12700.000000,377.040000,6.500000e+04,NaN,17.820000,0.000000,...,3.0,120.210000,3.0,13.000000,352.815000,10184.380000,144.010000,4248.000000,45.000000,12.000000
75%,122155.000000,NaN,20000.000000,20000.000000,20000.000000,587.340000,9.200000e+04,NaN,24.420000,0.000000,...,3.0,205.280000,3.0,21.000000,602.970000,15778.330000,309.570000,6855.900000,50.000000,18.000000
max,421092.000000,NaN,40000.000000,40000.000000,40000.000000,1719.830000,9.300000e+06,NaN,999.000000,21.000000,...,3.0,828.490000,3.0,30.000000,2066.880000,36734.040000,1377.170000,28000.000000,93.990000,65.000000


Below is a short description of columns in the accepted loan data set.

In [8]:
df_features_desc(df_loan_accepted)

,Feature,Data Type,Description
0,id,object,A unique LC assigned ID for the loan listing.
1,member_id,float64,A unique LC assigned Id for the borrower member.
2,loan_amnt,float64,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
3,funded_amnt,float64,The total amount committed to that loan at that point in time.
4,funded_amnt_inv,float64,The total amount committed by investors for that loan at that point in time.
5,term,object,The number of payments on the loan. Values are in months and can be either 36 or 60.
6,int_rate,object,Interest Rate on the loan
7,installment,float64,The monthly payment owed by the borrower if the loan originates.
8,grade,object,LC assigned loan grade
9,sub_grade,object,LC assigned loan subgrade


## Rejected Loans

We load all loans rejected (not funded) on the LendingClub marketplace from 2007 to 2018 Q3 for a period of 11.5 years into the dataframe `df_loan_rejected`.

In [356]:
df_loan_rejected = pd.read_csv('data/RejectStatsA.csv', skiprows=(1))

In [269]:
df_loan_rejected.shape

(755491, 9)

There are approximately 22 millions of rejected loans, each of them has 9 features.

In [270]:
df_loan_rejected.describe()

,Amount Requested,Risk_Score,Policy Code
count,7.554910e+05,731562.000000,755491.0
mean,1.291072e+04,590.995754,0.0
std,1.068035e+04,179.254816,0.0
min,0.000000e+00,0.000000,0.0
25%,5.000000e+03,571.000000,0.0
50%,1.000000e+04,644.000000,0.0
75%,2.000000e+04,685.000000,0.0
max,1.400000e+06,850.000000,0.0


## Census Data
The United States Census Bureau provides data about the American people and economy. We have social, economic, and geographic data from 2016 grouped by zip code provided via Piazza by CS109a instructors.

We load census data from 2016 into the dataframe `df_census`. 


In [10]:
df_census = pd.read_csv('data/zipcode_demographics_2016_USA.csv')

In [11]:
df_census.shape

(33120, 135)

There are 33120 zip codes, each of them described by 135 statistics.

In [12]:
df_census.describe()

,Unnamed: 0,Population,zip code tabulation area,owner_renter_total,owner_occupied,renter_occupied,abroad_year_ago_total,abroad_year_ago_puerto_rico,abroad_year_ago_us_islands,abroad_year_ago_foreign,...,60_to_75k_2016_pct,75_to_100k_2016_pct,100_to_150k_2016_pct,150_to_200k_2016_pct,over_200k_2016_pct,No_Diploma_pct,High_school_pct,Some_college_pct,Bachelors_Degree_pct,Graduate_Degree_pct
count,33120.000000,33120.000000,33120.000000,32989.000000,32989.000000,32989.000000,32989.000000,32989.000000,32989.000000,32989.000000,...,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,32765.000000,32765.000000,32765.000000,32765.000000,32765.000000
mean,16559.500000,9724.409300,49666.334209,9298.151808,6073.192549,3224.959259,61.003031,2.476250,0.529843,57.996938,...,0.098562,0.107134,0.097093,0.034172,0.032929,0.010476,0.290389,0.232441,0.148160,0.084170
std,9561.064794,14358.657599,27564.925769,13855.431955,8730.614992,6313.579477,170.145836,21.042147,5.755175,164.081808,...,0.180024,0.188002,0.176695,0.105539,0.109215,0.019820,0.119239,0.091032,0.099925,0.086035
min,0.000000,0.000000,601.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8279.750000,718.000000,26634.750000,667.000000,501.000000,121.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.218567,0.186317,0.081940,0.032538
50%,16559.500000,2807.500000,49739.000000,2645.000000,1984.000000,538.000000,0.000000,0.000000,0.000000,0.000000,...,0.033608,0.039894,0.018367,0.000000,0.000000,0.005961,0.288293,0.230056,0.127752,0.058981
75%,24839.250000,13177.750000,72123.500000,12484.000000,8334.000000,3206.000000,35.000000,0.000000,0.000000,33.000000,...,0.119956,0.133992,0.125463,0.022970,0.011533,0.013532,0.357277,0.273011,0.196078,0.106354
max,33119.000000,115104.000000,99929.000000,113403.000000,81331.000000,87101.000000,3661.000000,1205.000000,292.000000,3661.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Loan Acceptance Data Cleaning

In this section we describe basic operations for cleaning our dataset.


In [494]:
df_loan_accepted_cleaned = df_loan_accepted.copy()

### Remove index column

In [495]:
df_loan_accepted_cleaned.drop(columns=['Unnamed: 0'], inplace=True)

### Remove columns with a constant value

In [496]:
cols_unique = list(df_loan_accepted_cleaned.columns[df_loan_accepted_cleaned.nunique()==1])
print('The following features with constant value were removed', cols_unique)
df_loan_accepted_cleaned.drop(cols_unique, axis=1, inplace=True)
print('{} features left, out of {} in the original dataset'.format(df_loan_accepted_cleaned.shape[1], df_loan_accepted.shape[1]))

The following features with constant value were removed ['policy_code', 'hardship_type', 'deferral_term', 'hardship_length']
141 features left, out of 146 in the original dataset


### Remove columns associated with over 90% missing values

In [497]:
df_missing = (df_loan_accepted_cleaned.isnull().sum()/df_loan_accepted_cleaned.shape[0]).to_frame('perc_missing').reset_index()
cols_missing = list(df_missing[df_missing.perc_missing>0.9]['index'])
print('The following features with over 90% missing values were removed.')
print(df_missing[df_missing.perc_missing>0.9])
df_loan_accepted_cleaned.drop(cols_missing, axis=1, inplace=True)
print('')
print('{} features left, out of {} in the original dataset'.format(df_loan_accepted_cleaned.shape[1], df_loan_accepted.shape[1]))

The following features with over 90% missing values were removed.
                                          index  perc_missing
0    id                                          0.999990    
1    member_id                                   1.000000    
18   url                                         1.000000    
19   desc                                        0.936799    
52   annual_inc_joint                            0.957427    
53   dti_joint                                   0.957427    
54   verification_status_joint                   0.957971    
110  revol_bal_joint                             0.963744    
111  sec_app_earliest_cr_line                    0.963744    
112  sec_app_inq_last_6mths                      0.963744    
113  sec_app_mort_acc                            0.963744    
114  sec_app_open_acc                            0.963744    
115  sec_app_revol_util                          0.964383    
116  sec_app_open_act_il                         0.963744    
117 

### Remove duplicate rows

In [498]:
orig_rows_count = df_loan_accepted_cleaned.shape[0]
df_loan_accepted_cleaned.drop_duplicates(inplace=True)
print("{} duplicated rows were removed.".format(orig_rows_count-df_loan_accepted_cleaned.shape[0]))

1 duplicated rows were removed.


### Remove empty rows

In [499]:
orig_rows_count = df_loan_accepted_cleaned.shape[0]
df_loan_accepted_cleaned.dropna(inplace=True, how='all')
print("{} empty rows were removed.".format(orig_rows_count-df_loan_accepted_cleaned.shape[0]))

1 empty rows were removed.


# More Data Cleaning

## Variables Groups

We divide the columns in the following groups:

- **loan data**: information about the loan at the moment when it was requested
- **loan followup**: information about the loan's followup throughout its term
- **borrower demographics**: information about the borrower
- **borrower financial profile**: financial background of the borrower at the moment when he requested the loan
- **borrower financial profile followup**: changes in financial profile of the borrower throughout the loan term

In [500]:
cols_loan_data = ['loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade','issue_d','loan_status','purpose','title','initial_list_status','application_type','disbursement_method']
cols_loan_followup = ['out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','debt_settlement_flag','pymnt_plan']
cols_borrower_demographics = ['emp_title','emp_length','home_ownership','annual_inc','verification_status', 'zip_code','addr_state']
cols_borrower_finance_profile = ['dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','tot_coll_amt','mort_acc','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','hardship_flag']
cols_borrower_finance_profile_followup = ['last_credit_pull_d','collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_cur_bal','open_acc_6m','open_act_il','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m']

## Loan Data

In this section we manually clean-up come features describing the resquest of loan by borrowers.

**Issue date**

Let's convert the loan's issue date into a datatime type and add month and quarter.

In [501]:
df_loan_accepted_cleaned['issue_q'] = pd.to_datetime(df_loan_accepted_cleaned.issue_d, format='%b-%Y').dt.to_period('Q')
df_loan_accepted_cleaned['issue_m'] = df_loan_accepted_cleaned.issue_d.str.replace(r'-\d+', '')

**Loan status**

Delinquency happens when a borrower fails to pay the minimum amount for an outstanding debt. In the countplot below we can see the amount of loans that incurred in any stage of delinquency, according to the definitions used by Lending Club.

    Charged Off — defaulted loans for which there is no expectation from the lender in recovering the debt
    Default — borrower has failed to pay his obligations for more than 120 days
    Late — borrower has failed to pay his obligations for 31 to 120 days
    Grace Period — borrower still has time to pay his obligations without being considered delinquent
    Late — payment is late by 16 to 30 days
    
The count of loans within each stage is given below.

In [502]:
df_loan_accepted_cleaned.loan_status.value_counts()

Fully Paid                                             89859
Current                                                83297
Charged Off                                            22881
Late (31-120 days)                                     2255 
In Grace Period                                        1344 
Late (16-30 days)                                      498  
Does not meet the credit policy. Status:Fully Paid     198  
Does not meet the credit policy. Status:Charged Off    71   
Default                                                4    
Name: loan_status, dtype: int64

Very few old loans have the status 'Does not meet the credit policy' and will not be considered in our project.

In [503]:
df_loan_accepted_cleaned.drop(df_loan_accepted_cleaned[df_loan_accepted_cleaned.loan_status=='Does not meet the credit policy. Status:Fully Paid'].index, inplace=True)
df_loan_accepted_cleaned.drop(df_loan_accepted_cleaned[df_loan_accepted_cleaned.loan_status=='Does not meet the credit policy. Status:Charged Off'].index, inplace=True)

We add a new feature for successfully paid loans.

In [504]:
df_loan_accepted_cleaned['success'] = df_loan_accepted_cleaned['loan_status']
df_loan_accepted_cleaned.replace({'success':{'Charged Off': 0,
                                             'Fully Paid': 1, 
                                             'Current': 2, 
                                             'In Grace Period': 3,
                                             'Late (16-30 days)': 4,
                                             'Late (31-120 days)': 5,
                                             'Default': 6 }}, inplace=True)
df_loan_accepted_cleaned['success'] = df_loan_accepted_cleaned['success'].astype('int')

**Term and interest rate**

We turn the term and interest rate into numbers.

In [505]:
df_loan_accepted_cleaned.term.unique()

array([' 36 months', ' 60 months'], dtype=object)

In [506]:
df_loan_accepted_cleaned.term.replace(' 36 months', 36, inplace=True)
df_loan_accepted_cleaned.term.replace(' 60 months', 60, inplace=True)
df_loan_accepted_cleaned.term = df_loan_accepted_cleaned.term.astype('int')

In [507]:
df_loan_accepted_cleaned.term.unique()

array([36, 60], dtype=int64)

In [508]:
df_loan_accepted_cleaned.int_rate.head()

0     13.35%
1     14.08%
2     11.99%
3      8.18%
4     16.99%
Name: int_rate, dtype: object

In [509]:
df_loan_accepted_cleaned.int_rate = df_loan_accepted_cleaned.int_rate.str[:-1]
df_loan_accepted_cleaned.int_rate = df_loan_accepted_cleaned.int_rate.astype('float32')

In [510]:
df_loan_accepted_cleaned.int_rate.head()

0    13.35
1    14.08
2    11.99
3    8.18 
4    16.99
Name: int_rate, dtype: float32

**Loan amounts**

We will transform the amounts into integers

In [511]:
df_loan_accepted_cleaned.loan_amnt = df_loan_accepted_cleaned.loan_amnt.astype('int')

We will drop rows where `loan_amnt`, `funded_amnt`, `funded_amnt_inv` or `installment` is missing. We transform the amounts into integers.

In [512]:
df_loan_accepted_cleaned = df_loan_accepted_cleaned[df_loan_accepted_cleaned.loan_amnt.notnull() &
                                                    df_loan_accepted_cleaned.funded_amnt.notnull() & 
                                                    df_loan_accepted_cleaned.funded_amnt_inv.notnull() & 
                                                    df_loan_accepted_cleaned.installment.notnull()]

In [513]:
df_loan_accepted_cleaned.funded_amnt_inv = df_loan_accepted_cleaned.loan_amnt.astype('int')

**Title**

We will replace missing `title` values with NA for not available.

In [514]:
df_loan_accepted_cleaned.replace({'title': {np.nan: 'N/A'}}, inplace=True)

There are thousands of distinct titles entered by borrowers for their loan.

In [515]:
df_loan_accepted_cleaned.title.unique().shape

(8638,)

 We will prepare the titles for natural language features extration.

In [516]:
def clean_text(text):
    #https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
    # lower case
    text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    # remove punctuation
    text = text.str.replace('[^\w\s]','') 
    # remove stop words
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # correct spelling
    #from textblob import TextBlob
    #text = text.apply(lambda x: str(TextBlob(x).correct()))
    # lemmatization 
    from textblob import Word
    text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    return text

In [517]:
df_loan_accepted_cleaned.title = clean_text(df_loan_accepted_cleaned.title)

In [518]:
df_loan_accepted_cleaned.title.unique()

array(['car financing', '', 'debt consolidation', ..., 'envelope system',
       'medical bill debt consolidation', 'beckys wedding'], dtype=object)

**Miscellenous**

The columns `initial_list_status`,`application_type`,`disbursement_method` describe two categories each, one of them being largely dominant.

In [519]:
df_loan_accepted_cleaned.initial_list_status.value_counts()

w    130974
f    69164 
Name: initial_list_status, dtype: int64

In [520]:
df_loan_accepted_cleaned.application_type.value_counts()

Individual    191606
Joint App     8532  
Name: application_type, dtype: int64

In [521]:
df_loan_accepted_cleaned.disbursement_method.value_counts()

Cash         197328
DirectPay    2810  
Name: disbursement_method, dtype: int64

We remove those columns.

In [522]:
df_loan_accepted_cleaned.drop(columns=['initial_list_status','application_type','disbursement_method'], inplace=True)

## Loan Followup

In this section we manually clean-up the information about the current status of an active loan. These columns tell for example how much was paid back, when the next payment is to be expected etc.

For the goal of our project, we will consider `total_rec_late_fee` which indicate that the borrower had issues with paying the installment at some point in the past. This information could help computing the probability of charge-off or default for ongoing loans. 

We will also consider `last_pymnt_d`, `total_rec_prncp`, `total_rec_int`, `last_pymnt_d` for computing the return of investment for closed loans.

We delete the remaining follow-up columns.

In [523]:
df_loan_accepted_cleaned.drop(columns=['out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv',
                                       'last_pymnt_amnt','next_pymnt_d','debt_settlement_flag','pymnt_plan','recoveries','hardship_flag'], inplace=True)

## Borrower demographics

In this section we manually clean-up data related to the borrower.

**Employment Title**

We replace missing employment title by 'N/A'. 

In [524]:
df_loan_accepted_cleaned.replace({'emp_title': {np.nan: 'N/A'}}, inplace=True)

**Employment Length**

Missing employment length is replaced by 0.

In [525]:
df_loan_accepted_cleaned.emp_length.fillna(value=0,inplace=True)
df_loan_accepted_cleaned.emp_length.replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df_loan_accepted_cleaned.emp_length.replace(to_replace='self-employed', value='0', inplace=True, regex=True)
df_loan_accepted_cleaned.emp_length.replace(to_replace='', value='0', inplace=True, regex=True)
df_loan_accepted_cleaned.emp_length = df_loan_accepted_cleaned.emp_length.astype(int)

**Annual Income**

There are too many outliers in `annual_inc`, which should be removed.

In [526]:
orig_rows_count = df_loan_accepted_cleaned.shape[0]
df_loan_accepted_cleaned = df_loan_accepted_cleaned[~(df_loan_accepted_cleaned.annual_inc > 250000)]
print("{} rows removed with annual_inc > 250000.".format(orig_rows_count-df_loan_accepted_cleaned.shape[0]))

2213 rows removed with annual_inc > 250000.


## Borrower Financial Profile

In this section we clean columns which describe the credit history of the borrower.

**Revolving Line Utilization Rate**

We remove the '%' sign from the revolving line utilization rate and turn the column to float datatype.

In [527]:
df_loan_accepted_cleaned.revol_util = df_loan_accepted_cleaned.revol_util.fillna('0%')
df_loan_accepted_cleaned.revol_util = df_loan_accepted_cleaned.revol_util.str[:,-1]
df_loan_accepted_cleaned.revol_util = df_loan_accepted_cleaned.revol_util.astype('float32')

**Credit History**

It is safe to impute missing values for the following columns with zero.

In [528]:
cols = ['dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','tot_coll_amt','mort_acc','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']

df_loan_accepted_cleaned[cols] = df_loan_accepted_cleaned[cols].fillna(0)

**Trade Delinquency**

The percent of trades never delinquent `pct_tl_nvr_dlq` is set to 100% if it is missing.

In [529]:
df_loan_accepted_cleaned.pct_tl_nvr_dlq.fillna(100, inplace=True)

## Borrower Financial Profile Followup

In this section we manually clean some columns which contain a more current financial information about the borrower.

`last_credit_pull_d` indicates how old the financial information about the borrower is. It is safe to drop this feature.

In [530]:
df_loan_accepted_cleaned.drop(columns=['last_credit_pull_d'], inplace=True)

We will set the ratio of total current balance to high credit/credit limit for all bankcard accounts to 100% when missing.

In [531]:
df_loan_accepted_cleaned.bc_util.fillna(100, inplace=True)

It is safe to impute missing values for the remaining columns with zero.

In [532]:
cols = ['collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_cur_bal','open_acc_6m','open_act_il','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m']

df_loan_accepted_cleaned[cols] = df_loan_accepted_cleaned[cols].fillna(0)

## Correlation

In this section, we check the correlation between all remaining features.

In [533]:
def find_high_correlated_features(frame):
    new_corr = frame.corr()
    new_corr.loc[:,:] = np.tril(new_corr, k=-1) 
    new_corr = new_corr.stack()
    print(new_corr[(new_corr > 0.8) | (new_corr < -0.8)])


In [534]:
find_high_correlated_features(df_loan_accepted_cleaned)   

funded_amnt                 loan_amnt            0.999743
funded_amnt_inv             loan_amnt            1.000000
                            funded_amnt          0.999743
installment                 loan_amnt            0.945965
                            funded_amnt          0.946320
                            funded_amnt_inv      0.945965
open_rv_24m                 open_rv_12m          0.834431
all_util                    il_util              0.828947
avg_cur_bal                 tot_cur_bal          0.823542
num_actv_rev_tl             num_actv_bc_tl       0.830213
num_bc_sats                 num_actv_bc_tl       0.842139
num_op_rev_tl               open_acc             0.801644
                            num_actv_rev_tl      0.817740
num_rev_accts               num_bc_tl            0.853224
                            num_op_rev_tl        0.808960
num_rev_tl_bal_gt_0         num_actv_bc_tl       0.824096
                            num_actv_rev_tl      0.984166
              

As shown above, `installment` carries the same information as the `funded_amnt_in`. Similarly `open_il_24m` and `open_il_12m` are highly correlated. Same for `open_rv_24m` and `open_rv_12m`; `all_util` and `il_util`; `total_rev_hi_lim` and `revol_bal`; `bc_util` and `revol_util`; `avg_cur_bal` and `tot_cur_bal`; `num_actv_bc_tl` and `num_actv_bc_tl`; `num_tl_30dpd` and `acc_now_delinq`. We consider dropping some of those columns below.

In [535]:
df_loan_accepted_cleaned.drop(columns=['open_il_12m','open_rv_12m','il_util','revol_bal', 
                                       'revol_util','avg_cur_bal','num_actv_bc_tl','num_tl_30dpd'], inplace=True)

## Census Data

After cleaning the loan acceptance data, we will now clean and add census data to it.

### Cleaning census data

In [536]:
df_census_cleaned = df_census.copy()

We keep the following columns and remove the rest from the census data.

In [537]:
census_cols = ['Population', 'zip code tabulation area', 'median_income_2016', 
               'male_pct', 'female_pct', 
               'Black_pct', 'Native_pct', 'Asian_pct', 'Hispanic_pct', 
               'household_family_pct', 'poverty_level_below_pct', 'Graduate_Degree_pct', 'employment_2016_rate']
df_census_cleaned = df_census_cleaned[census_cols]

We remove rows with median income less than zero.

In [538]:
df_census_cleaned = df_census_cleaned[df_census_cleaned.median_income_2016>0]

We first fill missing values with zeros in the census dataset.

In [539]:
df_census_cleaned.isnull().sum()

Population                  0   
zip code tabulation area    0   
median_income_2016          0   
male_pct                    0   
female_pct                  0   
Black_pct                   0   
Native_pct                  0   
Asian_pct                   0   
Hispanic_pct                0   
household_family_pct        0   
poverty_level_below_pct     9760
Graduate_Degree_pct         0   
employment_2016_rate        0   
dtype: int64

In [540]:
df_census_cleaned.fillna(0, inplace=True)

Add a new column with zip codes in the format 123XX

In [541]:
df_census_cleaned['zip_code'] = df_census_cleaned['zip code tabulation area'].astype('str')
df_census_cleaned['zip_code'] = df_census_cleaned['zip_code'].str.pad(5, 'left', '0')
df_census_cleaned['zip_code'] = df_census_cleaned['zip_code'].str.slice(0,3)
df_census_cleaned['zip_code'] = df_census_cleaned['zip_code'].str.pad(5, 'right', 'x')
df_census_cleaned.drop(columns=['zip code tabulation area'], inplace=True)

Aggregate by zipcode and take the mean of census values

In [542]:
df_census_cleaned = df_census_cleaned.groupby(['zip_code']).mean().reset_index()

In [543]:
df_census_cleaned.tail()

,zip_code,Population,median_income_2016,male_pct,female_pct,Black_pct,Native_pct,Asian_pct,Hispanic_pct,household_family_pct,poverty_level_below_pct,Graduate_Degree_pct,employment_2016_rate
884,995xx,6268.461538,59252.903846,0.526823,0.473177,0.020715,0.459778,0.040214,0.049570,0.684839,0.063769,0.069403,0.760864
885,996xx,2560.727273,50849.181818,0.534283,0.465717,0.007251,0.565686,0.034245,0.027964,0.695934,0.089168,0.046758,0.749790
886,997xx,2019.910448,48305.447761,0.546723,0.453277,0.007017,0.652971,0.010724,0.022896,0.697308,0.029398,0.047139,0.733550
887,998xx,3639.714286,61540.857143,0.518490,0.481510,0.015683,0.196095,0.031797,0.044007,0.663403,0.036125,0.076276,0.829295
888,999xx,2420.888889,49593.444444,0.532552,0.467448,0.001796,0.285796,0.015533,0.024939,0.558452,0.123077,0.061948,0.822142


### Loan and census data consolidation

In [544]:
 df_loan_accepted_census_cleaned = pd.merge(df_loan_accepted_cleaned, df_census_cleaned, on='zip_code')

In [545]:
print('The merged loan and census dataset has {} features'.format(df_loan_accepted_census_cleaned.shape[1]))

The merged loan and census dataset has 99 features


Let's save the final accepted loan dataset augmented with census data to disk.

Merge accepted loan data with census data using the zip code

In [546]:
df_loan_accepted_census_cleaned.to_csv('df_loan_accepted_census_cleaned.csv')

# Loan Rejection Data Cleaning

In this section we will have a look at the rejected loan requests and do basic cleaning.


In [366]:
df_loan_rejected.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0


In [383]:
df_loan_rejected_cleaned = df_loan_rejected.copy()

We rename the columns in order to be able to merge the data with accepted loans later.

In [384]:
df_loan_rejected_cleaned.rename(columns={'Amount Requested':'loan_amnt',
                                         'Application Date': 'issue_d',
                                         'Risk_Score': 'risk_score',
                                         'Debt-To-Income Ratio': 'dti',
                                         'Zip Code':'zip_code',
                                         'State': 'addr_state',
                                         'Employment Length': 'emp_length',
                                         'Loan Title': 'title'},  inplace=True)

We remove columns with constant value

In [385]:
cols_unique = list(df_loan_rejected_cleaned.columns[df_loan_rejected_cleaned.nunique()==1])
print('Following columns with constant value were removed.')
print(cols_unique)
df_loan_rejected_cleaned.drop(cols_unique, axis=1, inplace=True)

Following columns with constant value were removed.
['Policy Code']


We remove duplicated rows

In [386]:
orig_rows_count = df_loan_rejected_cleaned.shape[0]
df_loan_rejected_cleaned.drop_duplicates(inplace=True)
print("{} duplicated rows were removed.".format(orig_rows_count-df_loan_rejected_cleaned.shape[0]))

1379 duplicated rows were removed.


We remove rows with empty Risk Score, Zip Code or State

In [387]:
df_loan_rejected_cleaned = df_loan_rejected_cleaned[df_loan_rejected_cleaned.risk_score.notnull() & 
                                                    df_loan_rejected_cleaned.zip_code.notnull() & 
                                                    df_loan_rejected_cleaned.addr_state.notnull()]

We prepare the loan title for natural language feature extraction.

In [388]:
df_loan_rejected_cleaned.replace({'title': {np.nan: 'N/A'}}, inplace=True)
df_loan_rejected_cleaned.title = clean_text(df_loan_rejected_cleaned.title)

We transform employment length to integer.

In [389]:
df_loan_rejected_cleaned.emp_length.fillna(value=0,inplace=True)
df_loan_rejected_cleaned.emp_length.replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df_loan_rejected_cleaned.emp_length.replace(to_replace='', value='0', inplace=True, regex=True)
df_loan_rejected_cleaned.emp_length = df_loan_rejected_cleaned.emp_length.astype(int)

Transform debt to income ratio to float.

In [390]:
df_loan_rejected_cleaned.dti = df_loan_rejected_cleaned.dti.str.replace('%','')
df_loan_rejected_cleaned.dti = df_loan_rejected_cleaned.dti.astype('float32')

Add application month and quarter.

In [391]:
df_loan_rejected_cleaned['issue_m']= pd.to_datetime(df_loan_rejected_cleaned.issue_d, format='%Y-%m-%d').dt.strftime("%b")
df_loan_rejected_cleaned['issue_q']= pd.to_datetime(df_loan_rejected_cleaned.issue_d, format='%Y-%m-%d').dt.to_period('Q')

Loan and census data consolidation

In [392]:
 df_loan_rejected_census_cleaned = pd.merge(df_loan_rejected_cleaned, df_census_cleaned, on='zip_code')

In [393]:
df_loan_rejected_census_cleaned.head()

,loan_amnt,issue_d,title,risk_score,dti,zip_code,addr_state,emp_length,issue_m,issue_q,...,male_pct,female_pct,Black_pct,Native_pct,Asian_pct,Hispanic_pct,household_family_pct,poverty_level_below_pct,Graduate_Degree_pct,employment_2016_rate
0,1000.0,2007-05-26,wedding covered honeymoon,693.0,10.00,481xx,NM,4,May,2007Q2,...,0.494315,0.505685,0.071903,0.003197,0.032006,0.03931,0.671069,0.225222,0.127648,0.804903
1,17000.0,2007-06-18,caterik,628.0,22.76,481xx,MI,1,Jun,2007Q2,...,0.494315,0.505685,0.071903,0.003197,0.032006,0.03931,0.671069,0.225222,0.127648,0.804903
2,3000.0,2007-06-20,joelbacon,683.0,4.69,481xx,OH,2,Jun,2007Q2,...,0.494315,0.505685,0.071903,0.003197,0.032006,0.03931,0.671069,0.225222,0.127648,0.804903
3,2300.0,2007-07-13,chrissy,448.0,6.24,481xx,MI,9,Jul,2007Q3,...,0.494315,0.505685,0.071903,0.003197,0.032006,0.03931,0.671069,0.225222,0.127648,0.804903
4,8000.0,2007-09-04,secure216,468.0,19.73,481xx,MS,1,Sep,2007Q3,...,0.494315,0.505685,0.071903,0.003197,0.032006,0.03931,0.671069,0.225222,0.127648,0.804903


Let's save the final rejected loan dataset augmented with census data to disk.

In [394]:
df_loan_rejected_census_cleaned.to_csv('df_loan_rejected_census_cleaned.csv')